What all has been learnt
1. matplotplib subplotting
2. LightGB
3. Bimodal and use of Gmm


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split , cross_validate
from sklearn.ensemble import GradientBoostingClassifier, RandomForestRegressor
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LinearRegression
%matplotlib inline 

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/test.csv')
submission = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/sample_submission.csv')

In [ ]:
display(train.head())
train_y = train['target']
train_X = train.drop(['target', 'id'], axis=1)
display(train_X.shape)

In [ ]:
display(train.info())
display(train.describe())

In [ ]:
train.hist(figsize = (40,40));

In [ ]:
col = ['cont'+str(i) for i in range(1,15)]
fig,ax = plt.subplots(4,4, figsize=(40,40))
for c,i in zip(col[0:4],range(0,4)):
    sns.scatterplot(train[c],train['target'], ax=ax[0,i])
for c,i in zip(col[4:8],range(0,4)):
    sns.scatterplot(train[c],train['target'], ax=ax[1,i])
for c,i in zip(col[8:12],range(0,4)):
    sns.scatterplot(train[c],train['target'], ax=ax[2,i])
for c,i in zip(col[12:14],range(0,2)):
    sns.scatterplot(train[c],train['target'], ax=ax[3,i])

plt.show()

checking for outliers using Boxplots

In [ ]:
def dist_box():
    fig,ax = plt.subplots(15,3, figsize=(30,60))
    sns.distplot(train['target'], ax=ax[0, 0])
    sns.boxplot(train['target'], ax=ax[0,1], color="maroon", saturation=4.6)
    stats.probplot(train['target'], plot=ax[0,2])
    for c,i in zip(col, range(1,15)):
        sns.distplot(train[c], ax=ax[i,0])
        sns.boxplot(train[c], ax=ax[i,1], color="maroon", saturation=4.6)
        stats.probplot(train[c], plot=ax[i,2])
        
        
    plt.show()

In [ ]:
dist_box()

QQ plot

In [ ]:
corr=train.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
plt.figure(figsize=(15,10))
sns.heatmap(corr,cmap="RdPu", mask=mask, annot=True)

1. Deal with outliers
2. Deal with Bimodal distribution
3. other Feature engineering
4. choosing model

In [ ]:
train=train[train.target != 0]
sns.distplot(train['target'])
train.head()

for a, b in enumerate(['a','b']):
    print(a)
    print(b)
    iloc

# BaseModel RandomForest


1. train has all the data
2. train_X has the training features
3. train_Y has the target variable
4. need to divide to a validation and train feature
5. test data to be kept untouched

In [ ]:
X , test_X, y, test_y = train_test_split(train_X, train_y, test_size = 0.3, random_state=42)  

In [ ]:
For_Reg = RandomForestRegressor(n_estimators=20, max_depth = 14, random_state=420)
For_Reg.fit(X, y)
test_y_cap = For_Reg.predict(test_X)
print(mse(test_y_cap, test_y))

In [ ]:
test1 = test.drop('id', axis=1)
test_y_cap = For_Reg.predict(test1)

In [ ]:
s1 = submission.copy()
s1['target'] = test_y_cap

In [ ]:
s1.to_csv("/kaggle/working/submission1.csv")

Random Forest Regression doesnt need the data to be normalized, like linear regression, the data we have is Bimodal, hence we will now try to feature engineer the data and make it suitable for lin_reg, and ensemble with the random forest regressor

In [ ]:
def get_gmm_class_feature(feat, n):
    gmm = GaussianMixture(n_components=n, random_state=42)

    gmm.fit(train[feat].values.reshape(-1, 1))

    train[f'{feat}_class'] = gmm.predict(train[feat].values.reshape(-1, 1))
    test[f'{feat}_class'] = gmm.predict(test[feat].values.reshape(-1, 1))

get_gmm_class_feature('cont1', 4)
get_gmm_class_feature('cont2', 10)
get_gmm_class_feature('cont3', 6)
get_gmm_class_feature('cont4', 4)
get_gmm_class_feature('cont5', 3)
get_gmm_class_feature('cont6', 2)
get_gmm_class_feature('cont7', 3)
get_gmm_class_feature('cont8', 4)
get_gmm_class_feature('cont9', 4)
get_gmm_class_feature('cont10', 8)
get_gmm_class_feature('cont11', 5)
get_gmm_class_feature('cont12', 4)
get_gmm_class_feature('cont13', 6)
get_gmm_class_feature('cont14', 6)

In [ ]:
display(train.head())
display(test.head())

In [ ]:
train_y = train['target']
train_X = train.drop(['target', 'id'], axis=1)
X , test_X, y, test_y = train_test_split(train_X, train_y, test_size = 0.3, random_state=42)  
test2 = test.drop('id', axis=1)

In [ ]:
Lin_Reg = LinearRegression()
Lin_Reg.fit(X,y)
test_y_cap2 = Lin_Reg.predict(test_X)
print(mse(test_y_cap2, test_y))


In [ ]:
test_y_cap2 = Lin_Reg.predict(test2)

In [ ]:
print(type(test_y_cap2))
print(type(test_y_cap))
res = (test_y_cap + test_y_cap2)/2

In [ ]:
s2 = submission.copy()
s2['target'] = test_y_cap
s2.to_csv("/kaggle/working/submission2.csv")